"people from different cultures can agree that a song is angry, but can differ on whether that feeling is positive or negative" - some random study I didn't citeimsosry
"creativity is the residue of wasted time" - Einstein

In [22]:
# valence -> culture specific; positive or negative values

In [10]:
!python -m pip install --upgrade pip

     |████████████████████████████████| 2.1MB 572kB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3


In [12]:
!pip install --upgrade google-cloud
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 KB 2.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 236.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 KB 279.7 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.8/960.8 KB 333.9 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.12.2
    Uninstalling protobuf-3.12.2:
      Successfully uninstalled protobuf-3.12.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.29.0
    Uninstalling grpcio-1.29.0:
      Successfully uninstalled grpcio-1.29.0


In [16]:
import pandas as pd
import json
from google.cloud import bigquery
import os

In [20]:
# not secure but I'm lazy, steal what you must
home = os.path.expanduser('~')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f"{home}/.config/gcloud/alpine-beacon-336222-c6a1c087e3ef.json"

In [5]:
with open("data/mpd.slice.54000-54999.json", "r") as the_jfile:
    data = json.load(the_jfile)

for KEY, VAL in data["playlists"][1].items():
    print(f"{KEY} : {VAL}")

name : Girl Music
collaborative : false
pid : 54001
modified_at : 1487030400
num_tracks : 103
num_albums : 81
num_followers : 1
tracks : [{'pos': 0, 'artist_name': 'Missy Higgins', 'track_uri': 'spotify:track:1SyJOVMjGCY8m9IfD8YErg', 'artist_uri': 'spotify:artist:6aP8ry8w3bSNyfyqhE8rGf', 'track_name': 'Where I Stood', 'album_uri': 'spotify:album:5a0yup4Q20b2TxoW5dWKI3', 'duration_ms': 259120, 'album_name': 'On A Clear Night'}, {'pos': 1, 'artist_name': 'Kelly Clarkson', 'track_uri': 'spotify:track:5GsR58FcuUEAFdgxa4NKUu', 'artist_uri': 'spotify:artist:3BmGtnKgCSGYIUhmivXKWX', 'track_name': 'Beautiful Disaster - Live', 'album_uri': 'spotify:album:20jYcJane0oI7VoMNoEOJU', 'duration_ms': 274600, 'album_name': 'Breakaway'}, {'pos': 2, 'artist_name': 'Sara Bareilles', 'track_uri': 'spotify:track:7hEbLvHSHpmMZXuuw2nmmH', 'artist_uri': 'spotify:artist:2Sqr0DXoaYABbjBo9HaMkM', 'track_name': 'Uncharted', 'album_uri': 'spotify:album:627ukPRwYxyBREHxBq0vGJ', 'duration_ms': 214893, 'album_name': '

In [23]:
# Construct a BigQuery client object.
client = bigquery.Client()

# Set table_id to the ID of the table to create.
table_id = "alpine-beacon-336222.music.playlists" # "project.dataset.table_name"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("post_abbr", "STRING"),
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = "gs://cloud-samples-data/bigquery/us-states/us-states.csv"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

NotFound: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/alpine-beacon-336222/jobs?prettyPrint=false: Not found: Dataset alpine-beacon-336222:music